# pyphot demonstration

In [25]:
"""Load necessary packages"""
import pyphot #synthetic photometry
from astropy.table import Table #synthetic photometry
import numpy as np #input/output
from astropy.io import fits #input/output
from matplotlib import pyplot as plt #visualisation
pkgdir = '/usr/local/lib/python2.7/site-packages/pyphot/' #Edit this depending on user
libsdir = '/usr/local/lib/python2.7/site-packages/pyphot/libs/' #Edit this depending on user
demodir = '/Users/sundar/work/pyphot/pyphot/demo/' #Edit this depending on user

### pyphot defines two classes: Filter and Library. Each instance of the Filter class contains transmission information for one filter, and each instance of the Library class consists of a collection of Filter classes.

### Libraries are stored in the form of HDF5 files.

#### Load/view information from a library

In [26]:
"""Load entire HDF5 library"""
libraryName=pkgdir+'libs/synphot_PhIReSSTARTer.hd5'
filterLibrary = pyphot.get_library(fname=libraryName)

In [27]:
"""Names of the first ten filters available in the library"""
filterNames = filterLibrary.get_library_content()
for i in range(9):
    print filterNames[i]

HST_WFPC2_F791W
HST_WFC3_F621M
HST_WFC3_F125W
ans_3300
steward_h
HST_ACS_WFC_F606W
steward_j
steward_k
HST_WFC3_F680N


#### Load/view information for a (set of) filter(s)

In [28]:
"""Load information for a single filter"""
ans_3300 = filterLibrary.load_filters(['ans_3300']) #result is a list, even for a single filter!
"""Load information for a list of filters"""
filters = filterLibrary.load_filters(['ans_3300', 'steward_k'])

In [29]:
"""View filter information"""
ans_3300[0].info()

Filter object information:
    name:                 ans_3300
    detector type:        photon
    wavelength units:     AA
    central wavelength:   3292.836737 angstrom
    pivot wavelength:     3292.709202 angstrom
    effective wavelength: 3292.623065 angstrom
    norm:                 36.965000
    definition contains 23 points

    Zeropoints
        Vega: 21.196181 mag,
              3.32297779849e-09 erg / angstrom * centimeter ** 2 * second,
              1201.68535663 Jy
          AB: 19.995715 mag,
              1.00395430898e-08 erg / angstrom * centimeter ** 2 * second,
              3630.59058769 Jy
          ST: 21.100000 mag,
              3.6307805477e-09 erg / angstrom * centimeter ** 2 * second,
              1312.99577725 Jy
        


#### The raw information required to define a filter is the wavelength (WITH UNITS!), transmission, and the detector type (i.e., whether it is a "photon" or "energy" counter). All the other infomation above is automatically computed for each filter.

### Users can define their own filters using the Filter class, and use them either temporarily or store them permanently in a Library class.

# Computing synthetic photometry using pyphot

In [31]:
"""We will use the same set of filters for both examples below"""
filterNames = ['MCPS_U', 'MCPS_B', 'MCPS_U', 'MCPS_I', '2MASS_J', '2MASS_H', '2MASS_Ks',
                   'SPITZER_IRAC_36', 'SPITZER_IRAC_45', 'SPITZER_IRAC_58', 'SPITZER_IRAC_80',
                   'SPITZER_MIPS_24']
libraryName = libsdir+'synphot_PhIReSSTARTer.hd5'
filterLibrary = pyphot.get_library(fname=libraryName)

### Example 1: computing synthetic photometry for a single spectrum (Source: IRC+10216, instrument: ISO-SWS).
#### (Uses the extractPhotometry method available in helpers.py)

In [33]:
#Input spectrum
spec = np.loadtxt(demodir+'IRC+10216_ISO_SWS.dat', comments='#', usecols=(0,1))
#Retrieve filter responses interpolated over input wavelength grid
filterList = filterLibrary.load_filters(filterNames, interp=True, lamb=spec[:,0]*pyphot.unit['micron'])

In [34]:
#Upon execution, cls contains the central wavelengths for each filter, and sed the
#   synthetic photometry in that filter.
cls, sed = pyphot.extractPhotometry(spec[:,0], spec[:,1], filterList, Fnu=True, absFlux=False)
lamCen = np.array([a.magnitude for a in cls])

Photometry:|##########| 12/12 100% [time: 00:00, eta: 00:00, 3598.20 iters/sec]


In [35]:
#Plot results
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(spec[:,0], spec[:,1], '--', label='ISO SWS spectrum')
ax.plot(lamCen, sed, 'o', color='red', label='Synthetic photometry')
ax.set_xlabel('Wavelength (micron)')
ax.set_ylabel('Flux (Jy)', rotation=90.)
for i in range(len(filterNames)):
    ax.text(lamCen[i], sed[i]*1.2, filterNames[i], horizontalalignment='center', verticalalignment='bottom', fontsize=8, rotation=90.)
plt.title('Example 1: IRC+10216')
plt.show()


### Example 2: computing synthetic photometry for the entire GRAMS carbon-star grid.
#### (Users the GRAMS carbon-star grid FITS file, grams_c.fits.)

In [36]:
"""Read in the GRAMS C-star grid"""
"""IGNORE WARNINGS"""
hdulist = fits.open(demodir+'grams_c.fits')
cg = hdulist[1].data
hdulist.close()
inlam = cg['LSPEC'][0] #input wavelength in micron
infnu = cg['FSPEC'] #input flux grid in Jy
#Retrieve filter responses interpolated over input wavelength grid
filterList = filterLibrary.load_filters(filterNames, interp=True, lamb=inlam*pyphot.unit['micron'])

 ------------------------------------------------------------------------------- [astropy.io.fits.card]
 The GRAMS carbon-star grid (Srinivasan+, 2011)                                  [astropy.io.fits.card]
 Grid of dust radiative transfer models for O-rich AGB and RSG stars             [astropy.io.fits.card]
 NUMBER OF MODELS IN GRID: 12243                                                 [astropy.io.fits.card]
 PHOTOSPHERES: The Z/Z_sun=0.33 models of                                        [astropy.io.fits.card]
 Aringer et al. 2009 A&A 503, 913                                                [astropy.io.fits.card]
 DUST: amorphous carbon (ACAR, Zubko et al. 1996 MNRAS 282, 1321) with 10% of Si [astropy.io.fits.card]
 (Pegourie 1988 A&A 194, 335) by mass.                                           [astropy.io.fits.card]
 GRAIN SIZE DISTRIBUTION: KMH (Kim et al. 1994 ApJ 422, 164) with size>0.01 \mu  [astropy.io.fits.card]
 WAVELENGTH RANGE OF OUTPUT SPECTRA: ~0.23 to ~180 \mu m        

In [ ]:
#Upon execution, cls contains the central wavelengths for each filter, and seds the
#   synthetic photometry for each input spectrum in that filter.
cls, seds = pyphot.extractSEDs(inlam, infnu, filterList, Fnu=True, absFlux=False)
lamCen = np.array([a.magnitude for a in cls])

Photometry:|##########| 12/12 100% [time: 00:00, eta: 00:00, 241.97 iters/sec] 


In [ ]:
#Plot results
f, axarr = plt.subplots(2, 2)
specnum = [0, 99, 10345, 3000]
ct=0
for i in range(2):
    for j in range(2):
        axarr[i,j].plot(inlam, infnu[specnum[ct],:], '--', label='GRAMS model spectrum #'+str(specnum[ct]+1))
        axarr[i,j].plot(lamCen, seds[specnum[ct],:], 'o', color='red', label='Synthetic photometry')
        if i==1:
            axarr[i,j].set_xlabel('Wavelength (micron)')
        if j==0:
            axarr[i,j].set_ylabel('Flux (Jy)', rotation=90.)
        #for k in range(len(filterNames)):
        #    axarr[i,j].text(lamCen[k], seds[specnum[i+j],k]/40, filterNames[k], 
        #            horizontalalignment='left', verticalalignment='bottom', 
        #                    fontsize=8, rotation=90.)
        axarr[i,j].set_xscale('log')
        axarr[i,j].set_yscale('log')
        axarr[i,j].set_title('Model spectrum #'+str(specnum[ct]+1), fontsize=10)
        ct = ct + 1
plt.setp([a.get_xticklabels() for a in axarr[0,:]], visible=False)
#plt.setp([a.get_yticklabels() for a in axarr[:,1]], visible=False)
plt.suptitle('Example 2: the GRAMS carbon-star grid', fontsize=12)
plt.show()